In [2]:
# ML Model Code
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor  # Import XGBoost
import joblib

# Load dataset
data = pd.read_csv('crop_yield.csv')

# Preprocess the dataset
data_encoded = pd.get_dummies(data, columns=['Crop', 'Season', 'State'], drop_first=True)

# Split the dataset into features and target variable
X = data_encoded.drop(columns=['Yield'])  # Features
y = data_encoded['Yield']                 # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
rf = RandomForestRegressor(n_estimators=200, max_depth=20, random_state=42)
xgboost = XGBRegressor(n_estimators=200, max_depth=10, learning_rate=0.1, random_state=42)

# Train both models
rf.fit(X_train, y_train)
xgboost.fit(X_train, y_train)

# Hybrid model structure
class HybridModel:
    def __init__(self, model1, model2):
        self.model1 = model1
        self.model2 = model2

    def predict(self, X):
        pred1 = self.model1.predict(X)
        pred2 = self.model2.predict(X)
        return (pred1 + pred2) / 2

# Save the hybrid model
hybrid_model = HybridModel(rf, xgboost)
joblib.dump(hybrid_model, 'crop_yield_hybrid_model.pkl')

print("\nHybrid model saved successfully as 'crop_yield_hybrid_model.pkl'.")



Hybrid model saved successfully as 'crop_yield_hybrid_model.pkl'.


In [3]:
# Uninstall the incorrect multipart package
!pip uninstall multipart -y

# Install the correct python-multipart package
!pip install python-multipart

# Upgrade Gradio to the latest version
!pip install --upgrade gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [4]:
import gradio as gr
import pandas as pd
import joblib

# Load the saved hybrid model
model = joblib.load('crop_yield_hybrid_model.pkl')

# Define the prediction function
def predict(crop, crop_year, season, state, area, production, annual_rainfall, fertilizer, pesticide):
    input_data = pd.DataFrame([{
        f"Crop_{crop}": 1,
        "Crop_Year": crop_year,
        f"Season_{season}": 1,
        f"State_{state}": 1,
        "Area": area,
        "Production": production,
        "Annual_Rainfall": annual_rainfall,
        "Fertilizer": fertilizer,
        "Pesticide": pesticide
    }])

    # Set missing dummy variables to 0 (for crops, states, and seasons not chosen)
    input_data = input_data.reindex(columns=X_train.columns, fill_value=0)

    prediction = model.predict(input_data)
    return prediction[0]

# Use dynamically extracted crops and states
crops = ['Arecanut', 'Arhar/Tur', 'Castor seed', 'Coconut ', 'Cotton(lint)', 'Dry chillies', 'Gram', 'Jute', 'Linseed', 'Maize']
states = ['Assam', 'Karnataka', 'Kerala', 'Meghalaya', 'West Bengal', 'Puducherry', 'Goa', 'Andhra Pradesh', 'Tamil Nadu', 'Odisha']
seasons = ['Whole Year', 'Kharif' , 'Rabi']

# Create Gradio interface
input_features = [
    gr.Dropdown(choices=crops, label="Crop"),
    gr.Slider(minimum=1990, maximum=2024, step=1, label="Crop Year"),
    gr.Dropdown(choices=seasons, label="Season"),
    gr.Dropdown(choices=states, label="State"),
    gr.Slider(minimum=0, maximum=10000000, label="Land Area (In Hectares)"),
    gr.Slider(minimum=0, maximum=10000000, label="Production"),
    gr.Slider(minimum=0, maximum=10000000, label="Annual Rainfall (mm)"),
    gr.Slider(minimum=0, maximum=10000000, label="Fertilizer (Kg per Hectare)"),
    gr.Slider(minimum=0, maximum=10000000, label="Pesticide (Litres per Hectare)")
]

output = gr.Number(label="Predicted Yield (Tons Per Hectare)")

# Custom CSS for setting background color to white
css = """
body {
    background-color: white;
}
"""

gr.Interface(fn=predict, inputs=input_features, outputs=output, css=css).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e04167b08c1b5b616b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
